In [ ]:
!pip install pulp

import pandas as pd
from pulp import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.2 MB 5.2 MB/s 


In [25]:
from pulp import *

In [1]:
import pandas as pd

In [2]:
# distance_matrix = pd.read_excel('./광주광역시_경계구역_읍면동_충장동통합.xlsx', usecols = 'B : GM')
distance_matrix = pd.read_excel('./광주광역시_경계구역_읍면동_충장동통합.xlsx')

In [3]:
pop = pd.read_excel('./행정안전부_지역별(법정동) 주민등록 인구수.xlsx')

In [4]:
pop_final = list(pop.iloc[:, 1])
final = distance_matrix[:194]

In [5]:
pop_final
final.columns

Index(['ID', '쌍촌동', '광천동', '덕흥동', '내방동', '양동', '금호동', '농성동', '화정동', '용두동',
       ...
       '매곡동', '동림동.1', '삼거동', '서석동', '일곡동', '연제동', '덕림동', '운암동', '충장동', '대의동'],
      dtype='object', length=194)

In [6]:
df = pop
df
pop_final

['대인동',
 '충장동',
 '수기동',
 '대의동',
 '궁동',
 '장동',
 '동명동',
 '계림동',
 '산수동',
 '지산동',
 '남동',
 '광산동',
 '금동',
 '호남동',
 '불로동',
 '황금동',
 '서석동',
 '소태동',
 '용연동',
 '운림동',
 '학동',
 '월남동',
 '선교동',
 '내남동',
 '용산동',
 '양동',
 '농성동',
 '광천동',
 '유촌동',
 '덕흥동',
 '쌍촌동',
 '화정동',
 '치평동',
 '내방동',
 '서창동',
 '세하동',
 '용두동',
 '풍암동',
 '벽진동',
 '금호동',
 '마륵동',
 '매월동',
 '동천동',
 '사동',
 '구동',
 '서동',
 '월산동',
 '백운동',
 '주월동',
 '노대동',
 '진월동',
 '덕남동',
 '행암동',
 '임암동',
 '송하동',
 '양림동',
 '방림동',
 '봉선동',
 '구소동',
 '양촌동',
 '도금동',
 '승촌동',
 '지석동',
 '압촌동',
 '화장동',
 '칠석동',
 '석정동',
 '신장동',
 '양과동',
 '이장동',
 '대지동',
 '원산동',
 '월성동',
 '중흥동',
 '유동',
 '누문동',
 '북동',
 '임동',
 '신안동',
 '용봉동',
 '동림동',
 '운암동',
 '우산동',
 '풍향동',
 '문흥동',
 '각화동',
 '두암동',
 '오치동',
 '삼각동',
 '매곡동',
 '충효동',
 '덕의동',
 '금곡동',
 '망월동',
 '청풍동',
 '화암동',
 '장등동',
 '운정동',
 '본촌동',
 '일곡동',
 '양산동',
 '연제동',
 '신용동',
 '용두동',
 '지야동',
 '태령동',
 '수곡동',
 '효령동',
 '용전동',
 '용강동',
 '생용동',
 '월출동',
 '대촌동',
 '오룡동',
 '송정동',
 '도산동',
 '신촌동',
 '서봉동',
 '운수동',
 '선암동',
 '소촌동',
 '우산동',
 '황룡동',
 '박호동',
 '비아동',
 '도천동',
 '수완

In [9]:
print(f'final의 column 수 : {len(final.columns)}')
print(f'pop_final의 columns 수 : {len(pop_final)}')

final의 column 수 : 194
pop_final의 columns 수 : 193


In [12]:
for i in range(194):
  for j in range(193):
    if(final.columns[i+1] == pop_final[j]):
      df['시군구명'][i] = pop['시군구명'][j]
      df['읍면동명'][i] = pop['읍면동명'][j]
      df['계'][i] = pop['계'][j]

C:\Users\user\AppData\Local\Temp\ipykernel_9000\1876391561.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['시군구명'][i] = pop['시군구명'][j]
C:\Users\user\AppData\Local\Temp\ipykernel_9000\1876391561.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['읍면동명'][i] = pop['읍면동명'][j]
C:\Users\user\AppData\Local\Temp\ipykernel_9000\1876391561.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['계'][i] = pop['계'][j]


IndexError: index 194 is out of bounds for axis 0 with size 194

In [ ]:
df

,시군구명,읍면동명,계
0,서구,쌍촌동,48520
1,서구,광천동,7805
2,서구,덕흥동,451
3,서구,내방동,7315
4,광산구,양동,86
...,...,...,...
188,광산구,소촌동,18240
189,광산구,소촌동,18240
190,광산구,두정동,174
191,서구,광천동,7805


In [ ]:
# distance_matrix, pop 행렬 순서 맞추기
pop_sort = []

for i in range(194):
  for j in range(194):


In [ ]:
final.set_index('ID', drop = True)

,쌍촌동,광천동,덕흥동,내방동,양동,금호동,농성동,화정동,용두동,노대동,...,매곡동,동림동.1,삼거동,서석동,일곡동,연제동,덕림동,운암동,충장동,대의동
ID,,,,,,,,,,,,,,,,,,,,,
쌍촌동,0.000000,2144.880040,2777.057990,1219.937464,3384.456895,2345.002069,2346.190337,1528.531870,7300.512049,7198.810977,...,4847.021614,3153.290374,17174.384225,6247.368919,7071.763137,5203.873351,17858.745719,3210.002830,4719.221324,5300.442001
광천동,2144.880040,0.000000,3978.915679,925.629677,2144.305975,4301.267261,1449.055894,2295.055030,9400.995362,7686.436453,...,2908.162558,2588.248015,18783.042094,5208.223763,5176.414398,4078.424700,19256.989765,1479.575465,3514.044542,4092.480465
덕흥동,2777.057990,3978.915679,0.000000,3325.467411,5880.422596,4103.830123,4880.983845,4304.702996,7631.902789,9827.266956,...,5500.964102,2650.422626,14809.415603,8897.869562,7306.402450,4488.934003,15297.367656,4042.530174,7278.975663,7871.125847
내방동,1219.937464,925.629677,3325.467411,0.000000,2557.305080,3440.043760,1586.586220,1718.937815,8497.535092,7430.223601,...,3705.422217,2613.922859,18057.813708,5603.358546,5960.689499,4450.337113,18616.163476,2120.308959,3959.531525,4551.738479
양동,3384.456895,2144.305975,5880.422596,2557.305080,0.000000,4696.017023,1038.669816,2359.309870,9784.527111,6063.669832,...,4137.886531,4723.348147,20544.220937,3077.121691,6210.027719,5981.641865,21158.755588,3323.836157,1404.414001,1995.914757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
연제동,5203.873351,4078.424700,4488.934003,4450.337113,5981.641865,7498.956207,5515.912989,6168.480218,11962.975428,11763.876336,...,2491.806174,2108.316689,17828.379195,8772.651572,3146.742693,0.000000,17758.223268,2667.898543,7132.787213,7625.168520
덕림동,17858.745719,19256.989765,15297.367656,18616.163476,21158.755588,17818.174018,20138.147663,19271.398776,16206.507848,23635.080169,...,19964.775300,17234.339096,2582.015981,24106.106011,20799.409150,17758.223268,0.000000,18973.207780,22544.151187,23133.617884
운암동,3210.002830,1479.575465,4042.530174,2120.308959,3323.836157,5530.965808,2872.202256,3738.373978,10481.191486,9136.349579,...,1653.432875,1753.938186,18697.397381,6217.197920,3862.574187,2667.898543,18973.207780,0.000000,4539.401996,5066.654206


In [ ]:
pop_final

In [ ]:
region_name = final.iloc[:1]
region_name_final = list(region_name)
location = region_name_final # 동 이름 전처리

In [ ]:
region_name_final

['ID',
 '쌍촌동',
 '광천동',
 '덕흥동',
 '내방동',
 '양동',
 '금호동',
 '농성동',
 '화정동',
 '용두동',
 '노대동',
 '치평동',
 '마륵동',
 '석정동',
 '신장동',
 '진월동',
 '방림동',
 '지석동',
 '압촌동',
 '이장동',
 '송하동',
 '대지동',
 '원산동',
 '주월동',
 '봉선동',
 '행암동',
 '임암동',
 '월성동',
 '덕남동',
 '양과동',
 '양림동',
 '화장동',
 '칠석동',
 '양촌동',
 '도금동',
 '승촌동',
 '구소동',
 '서동',
 '월산동',
 '사동',
 '구동',
 '운정동',
 '오룡동',
 '백운동',
 '장등동',
 '지야동',
 '태령동',
 '문흥동',
 '용두동.1',
 '용강동',
 '생용동',
 '수곡동',
 '효령동',
 '풍향동',
 '본촌동',
 '대촌동',
 '임동',
 '북동',
 '신안동',
 '유동',
 '누문동',
 '월출동',
 '각화동',
 '오치동',
 '용전동',
 '덕의동',
 '금곡동',
 '용봉동',
 '충효동',
 '화암동',
 '중흥동',
 '망월동',
 '청풍동',
 '우산동',
 '복룡동',
 '두암동',
 '신용동',
 '진곡동',
 '장덕동',
 '송대동',
 '안청동',
 '신룡동',
 '두정동',
 '산막동',
 '고룡동',
 '장수동',
 '산정동',
 '임곡동',
 '광산동',
 '옥동',
 '장록동',
 '월곡동',
 '등임동',
 '용동',
 '용곡동',
 '송촌동',
 '지죽동',
 '도덕동',
 '송정동',
 '지정동',
 '동산동',
 '신촌동',
 '소촌동',
 '도산동',
 '도호동',
 '박호동',
 '명화동',
 '우산동.1',
 '황룡동',
 '산월동',
 '신창동',
 '월계동',
 '쌍암동',
 '산수동',
 '선동',
 '신가동',
 '운남동',
 '본덕동',
 '용봉동.1',
 '오산동',
 '사호동',
 '지평동',
 '양동.1',
 '송산동',
 '오운동',
 '송학

In [ ]:
a = dict(zip(location, ))

In [ ]:
a = dict(zip(location, pop_final))

In [ ]:
a

{'각화동': '승촌동',
 '계림동': '오운동',
 '고룡동': '풍향동',
 '광산동': '오치동',
 '광산동.1': '선동',
 '광천동': '충장동',
 '구동': '금호동',
 '구소동': '세하동',
 '궁동': '용동',
 '금곡동': '칠석동',
 '금동': '송촌동',
 '금호동': '장동',
 '남동': '산수동',
 '남산동': '신가동',
 '내남동': '요기동',
 '내방동': '대의동',
 '내산동': '수완동',
 '노대동': '지산동',
 '농성동': '동명동',
 '누문동': '양촌동',
 '대산동': '월계동',
 '대의동': '도호동',
 '대인동': '월전동',
 '대지동': '학동',
 '대촌동': '송하동',
 '덕남동': '광천동',
 '덕림동': '양산동',
 '덕의동': '화장동',
 '덕흥동': '수기동',
 '도금동': '내방동',
 '도덕동': '장등동',
 '도산동': '신용동',
 '도천동': '하산동',
 '도호동': '용두동',
 '동림동': '흑석동',
 '동림동.1': '왕동',
 '동명동': '송산동',
 '동산동': '일곡동',
 '동천동': '삼도동',
 '동호동': '장수동',
 '두암동': '유동',
 '두정동': '운암동',
 '등임동': '덕의동',
 '마륵동': '광산동',
 '망월동': '대지동',
 '매곡동': '지산동',
 '매월동': '양동',
 '명도동': '안청동',
 '명화동': '태령동',
 '문흥동': '월산동',
 '박호동': '지야동',
 '방림동': '황금동',
 '백운동': '동천동',
 '벽진동': '송학동',
 '복룡동': '중흥동',
 '본덕동': '신촌동',
 '본촌동': '임암동',
 '봉선동': '내남동',
 '북동': '방림동',
 '북산동': '운남동',
 '불로동': '동산동',
 '비아동': '두정동',
 '사동': '벽진동',
 '사호동': '선암동',
 '산막동': '우산동',
 '산수동': '대촌동',
 '산수동.1': '도덕동',
 '

In [ ]:
D = {}
for i in range(0, 194): # 딕셔너리 형태로 한 동당 거리행렬 정의
    D[i] = dict(zip(location[i:], [dict(zip(location, list(final.iloc[i+1,1:])))]))

IndexError: ignored

In [26]:
merged = {}
for i in range(0, 97):
    merged = {**merged, **D[i]} # 424 + 424 거리행렬 정의

KeyError: 0

In [27]:
p = 26 # 임시선별검사소 수 정의

In [28]:
# P-Median 모델을 통해 최적지 선정하는 코드
X = LpVariable.dicts('X_%s_%s', (location, location), cat = 'Binary',
                     lowBound = 0,
                     upBound = 1)

In [29]:
prob = LpProblem('P Median', LpMinimize) # 목적 함수

C:\Users\user\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
prob += sum(sum(a[i]*distance_matrix[i][j] * X[i][j] for j in location)for i in location)
prob += sum(X[i][i] for i in location) == p

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


KeyError: ignored

In [ ]:
for i in location: prob += sum(X[i][j] for j in location) == 1 # 제약식
for i in location:
    for j in location: prob += X[i][j] <= X[j][j]

In [ ]:
prob.writeLP("p-median_Seoul_Whole.lp")
prob.solve()

In [ ]:
# distance_matrix = pd.read_excel('C:/Users/user/Downloads/광주광역시_읍면동_거리행렬.xlsx', usecols = 'B : GM')
# pop = pd.read_excel('C:/Users/user/Downloads/주민등록인구_20220804102055.xlsx', usecols = 'C')

pop_final = list(pop.iloc[0:97, 0])
final = distance_matrix[1:194] # 데이터 전처리 완료

region_name = final.iloc[0, 1:]
region_name_final = list(region_name)
location = region_name_final # 동 이름 전처리

a = dict(zip(location, pop_final))

D = {}
for i in range(0, 424): # 딕셔너리 형태로 한 동당 거리행렬 정의
    D[i] = dict(zip(location[i:], [dict(zip(location, list(final.iloc[i+1,1:])))]))
    
merged = {}
for i in range(0, 424):
    merged = {**merged, **D[i]} # 424 + 424 거리행렬 정의

p = 26 # 임시선별검사소 수 정의

# P-Median 모델을 통해 최적지 선정하는 코드
X = LpVariable.dicts('X_%s_%s', (location, location), cat = 'Binary',
                     lowBound = 0,
                     upBound = 1)

prob = LpProblem('P Median', LpMinimize) # 목적 함수
prob += sum(sum(a[i]*merged[i][j] * X[i][j] for j in location)for i in location)
prob += sum(X[i][i] for i in location) == p

for i in location: prob += sum(X[i][j] for j in location) == 1 # 제약식
for i in location:
    for j in location: prob += X[i][j] <= X[j][j]

prob.writeLP("p-median_Seoul_Whole.lp")
prob.solve()

IndexError: ignored

# p-median 알고리즘 논문


In [20]:
location = distance_matrix.columns

In [21]:
D = distance_matrix

In [22]:
p = 26

In [30]:
X = LpVariable.dicts('%s_%s', (location, location), cat = 'Binary', lowBound = 0, upBound = 1)
prob = LpProblem('P Median', LpMinimize)